In [7]:
!pip install sentencepiece nltk sacrebleu datasets evaluate


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [91]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import (
    MarianMTModel, 
    MarianTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)
from datasets import Dataset
import evaluate

import nltk
import sacrebleu
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

from tqdm import tqdm

In [2]:
# Загружаем необходимые данные для NLTK
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    print("Скачиваем данные NLTK для русского языка...")
    nltk.download('punkt_tab', quiet=False)
    nltk.download('punkt', quiet=False)

In [3]:
torch.cuda.is_available()

True

In [75]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "Helsinki-NLP/opus-mt-ru-en"
model = MarianMTModel.from_pretrained(model_name)
model.to(device)
tokenizer = MarianTokenizer.from_pretrained(model_name)

config.json: 0.00B [00:00, ?B/s]

D:\UI-agents\env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\slityagin\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-ru-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. F

pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

source.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/307M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

D:\UI-agents\env\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


## Подготовим данные

In [6]:
!git clone https://github.com/MicrosoftTranslator/NTREX.git

Cloning into 'NTREX'...


In [77]:
with open("NTREX/NTREX-128/newstest2019-src.eng.txt", 'r') as f:
    en_texts = f.read().split("\n")
    try:
        idx = en_texts.index("")
        en_texts.pop(idx)
    except Exception as e:
        pass
        
print(f"Кол-во англ. тектов: {len(en_texts)}")
print(f"Пример текста: {en_texts[0]}")

Кол-во англ. тектов: 1997
Пример текста: Welsh AMs worried about 'looking like muppets'


In [78]:
with open("NTREX/NTREX-128/newstest2019-ref.rus.txt", 'r', encoding='utf-8') as f:
    ru_texts = f.read().split("\n")
    try:
        idx = ru_texts.index("")
        ru_texts.pop(idx)
    except Exception as e:
        pass
        
print(f"Кол-во русских. тектов: {len(ru_texts)}")
print(f"Пример текста: {ru_texts[0]}")

Кол-во русских. тектов: 1997
Пример текста: Члены Ассамблеи Уэльса не хотят «походить на маппетов»


In [79]:
def split_train_val(en_texts, ru_texts, test_size=0.2, random_state=42):
    """
    Разделяет данные на обучающую и валидационную выборки
    """
    en_train, en_val, ru_train, ru_val = train_test_split(
        en_texts, 
        ru_texts, 
        test_size=test_size, 
        random_state=random_state,
        shuffle=True
    )
    
    print(f"Разделение данных:")
    print(f"  Train: {len(en_train)} примеров")
    print(f"  Val: {len(en_val)} примеров")
    
    return {
        'train': {'en': en_train, 'ru': ru_train},
        'val': {'en': en_val, 'ru': ru_val}
    }

In [80]:
split_data = split_train_val(en_texts, ru_texts, test_size=0.2)

Разделение данных:
  Train: 1597 примеров
  Val: 400 примеров


In [81]:
def prepare_dataset(en_texts, ru_texts, tokenizer, max_length=128):
    """
    Подготавливает данные в формате для обучения
    """
    # Формируем промпты для перевода на русский
    inputs = [f"translate to en: {text}" for text in ru_texts]
    targets = en_texts
    
    # Токенизация
    model_inputs = tokenizer(
        inputs,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    
    # Токенизация целей
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
    
    model_inputs["labels"] = labels["input_ids"]
    
    # Конвертируем в формат Dataset
    dataset_dict = {
        'input_ids': model_inputs['input_ids'],
        'attention_mask': model_inputs['attention_mask'],
        'labels': model_inputs['labels']
    }
    
    return Dataset.from_dict(dataset_dict)

# Создаем датасеты
train_dataset = prepare_dataset(
    split_data['train']['en'], 
    split_data['train']['ru'], 
    tokenizer
)

val_dataset = prepare_dataset(
    split_data['val']['en'], 
    split_data['val']['ru'], 
    tokenizer
)

print(f"\nДатасеты созданы:")
print(f"  Train dataset size: {len(train_dataset)}")
print(f"  Val dataset size: {len(val_dataset)}")


Датасеты созданы:
  Train dataset size: 1597
  Val dataset size: 400


## Чуть обучим модельку

In [92]:
def compute_metrics(eval_pred):
    """
    Вычисляет BLEU метрику для валидации
    Использует подход из вашего примера
    """
    predictions, labels = eval_pred
    
    # Если predictions в формате logits (batch_size, seq_len, vocab_size)
    if predictions.ndim == 3:
        # Берем argmax для получения токенов
        predictions = np.argmax(predictions, axis=-1)
    
    # Декодируем предсказания
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Заменяем -100 на pad_token_id в labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Очистка текстов
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]
    
    # Убираем пустые строки
    decoded_preds = [pred for pred in decoded_preds if pred]
    decoded_labels = [label for label in decoded_labels if label]
    
    if not decoded_preds or not decoded_labels:
        return {"bleu": 0.0}
    
    # Выравниваем длины
    min_len = min(len(decoded_preds), len(decoded_labels))
    decoded_preds = decoded_preds[:min_len]
    decoded_labels = decoded_labels[:min_len]
    
    # Подготавливаем данные для corpus_bleu
    # references: список списков списков [[['word1', 'word2', ...]], ...]
    references = [[ref.split()] for ref in decoded_labels]
    
    # hypotheses: список списков [['word1', 'word2', ...], ...]
    hypotheses = [hyp.split() for hyp in decoded_preds]
    
    # Вычисляем BLEU
    try:
        smooth = SmoothingFunction().method1
        bleu_score = corpus_bleu(
            references, 
            hypotheses, 
            smoothing_function=smooth
        )
        
        return {"bleu": bleu_score * 100}  # В процентах, как в примере
        
    except Exception as e:
        print(f"Ошибка при вычислении BLEU: {e}")
        # В случае ошибки показываем пример данных для отладки
        if decoded_preds and decoded_labels:
            print(f"Пример pred: {decoded_preds[0]}")
            print(f"Пример label: {decoded_labels[0]}")
        return None

In [97]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./mt-fine-tuned",
    
    # Стратегии оценки и сохранения
    eval_strategy="steps",  # исправлено: должно быть evaluation_strategy
    eval_steps=100,  # Раз в 100 шагов (чаще для 1.5к данных)
    save_strategy="steps",
    save_steps=100,
    
    # Логирование
    logging_dir="./logs",
    logging_steps=50,  # Логируем каждые 50 шагов
    report_to="none",  # Или "wandb" если используете
    
    # Параметры обучения для fine-tuning
    learning_rate=3e-5,  # Оптимально для fine-tuning
    per_device_train_batch_size=8,  # Можно увеличить с 4 до 8 для ускорения
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,  # Эффективный batch_size = 16
    
    # Ограничения
    max_steps=500,  # Ограничиваем шаги
    max_grad_norm=1.0,  # Клиппинг градиентов
    warmup_steps=50,  # Плавный старт
    
    # Детали генерации и оценки
    predict_with_generate=True,
    generation_max_length=128,
    generation_num_beams=4,
    
    # Ранняя остановка и сохранение лучшей модели
    load_best_model_at_end=True,
    metric_for_best_model="bleu",  # Лучше следить за loss для early stopping
    greater_is_better=True,
    
    # Дополнительные настройки
    weight_decay=0.01,  # Регуляризация
    fp16=torch.cuda.is_available(),  # Используем mixed precision если есть CUDA
    seed=42,
    dataloader_num_workers=2 if torch.cuda.is_available() else 0,
    
    # Для маленького датасета отключаем некоторые проверки
    no_cuda=not torch.cuda.is_available(),
    remove_unused_columns=True,  # Убираем неиспользуемые колонки
)

In [98]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,  # Игнорировать при вычислении loss
    pad_to_multiple_of=8 if training_args.fp16 else None,
)

In [99]:
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,  # Остановить после 3 эпох без улучшений
    early_stopping_threshold=0.01
)

In [102]:
model.train()
model.to(device)

print(f"Model is in training mode: {model.training}")
print(f"Trainable parameters: {sum(p.requires_grad for p in model.parameters())}/{len(list(model.parameters()))}")

# Если все параметры заморожены (маловероятно для MarianMT, но проверим)
if not any(p.requires_grad for p in model.parameters()):
    print("Размораживаем все параметры модели...")
    for param in model.parameters():
        param.requires_grad = True

Model is in training mode: True
Trainable parameters: 0/255
Размораживаем все параметры модели...


In [103]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

C:\Users\slityagin\AppData\Local\Temp\ipykernel_22380\1088118177.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [94]:
eval_results = trainer.evaluate()
print(f"BLEU score на валидации: {eval_results['eval_bleu']:.4f}")

BLEU score на валидации: 19.4605


In [104]:
# Запускаем обучение
train_result = trainer.train()

# Сохраняем метрики обучения
trainer.save_model()  # Сохраняем модель
trainer.log_metrics("train", train_result.metrics)
trainer.save_metrics("train", train_result.metrics)
trainer.save_state()

print(f"\nОбучение завершено!")
print(f"Total training time: {train_result.metrics.get('train_runtime', 0):.2f} seconds")
print(f"BLEU score на валидации: {eval_results['eval_bleu']:.4f}")

Step,Training Loss,Validation Loss,Bleu
100,0.441600,0.417150,22.643671
200,0.376000,0.409196,23.332018
300,0.318800,0.407410,23.421508
400,0.284400,0.407889,23.281784
500,0.261500,0.408612,23.374511


D:\UI-agents\env\Lib\site-packages\transformers\modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


***** train metrics *****
  epoch                    =        5.0
  total_flos               =   252088GF
  train_loss               =     0.4678
  train_runtime            = 0:12:32.34
  train_samples_per_second =     10.633
  train_steps_per_second   =      0.665

Обучение завершено!
Total training time: 752.35 seconds
BLEU score на валидации: 19.4605


In [105]:
eval_results = trainer.evaluate()
print(f"BLEU score на валидации: {eval_results['eval_bleu']:.4f}")

BLEU score на валидации: 23.4215


## Инференс

In [106]:
def translate_batch(model, sentences, max_length=128):
    model.eval()
    
    inputs = tokenizer(
            sentences,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length
        ).to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=max_length)

    translations = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return translations

In [107]:
translate_batch(model, ["Привет, я подсяду", "Привет, я отсяду?", "Пока, я улетел на воздушном шаре"])

["Hey, I'll take a seat.",
 'Hey, can I sit down?',
 'Bye, I took off in a balloon.']

In [10]:
class EmbeddingDataset(Dataset):
    """Датасет для уже извлеченных эмбеддингов"""
    def __init__(self, embeddings, labels):
        self.embeddings = torch.FloatTensor(embeddings)
        self.labels = torch.LongTensor(labels)
        
    def __len__(self):
        return len(self.embeddings)
    
    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

def create_balanced_dataloader(embeddings, labels, batch_size=32):
    """Создает DataLoader с балансировкой классов"""
    dataset = EmbeddingDataset(embeddings, labels)
    
    # Вычисляем веса для каждого класса
    class_counts = np.bincount(labels)
    class_weights = 1. / class_counts
    sample_weights = class_weights[labels]
    
    # Создаем сэмплер
    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True  # Разрешаем повторную выборку для малых классов
    )
    
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        sampler=sampler,  # используем sampler вместо shuffle
        drop_last=True
    )
    
    return dataloader

In [12]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(list(dataset_df["rating"]))  # Преобразуем 1-10 в 0-9

In [19]:
X_train, X_val, y_train, y_val = train_test_split(
        embeddings,
        labels_encoded,
        test_size=0.2,
        random_state=42,
        stratify=labels_encoded,
        shuffle=True
    )

print(f"Обучающая выборка: {len(X_train)} примеров")
print(f"Валидационная выборка: {len(X_val)} примеров")

Обучающая выборка: 86506 примеров
Валидационная выборка: 21627 примеров


In [20]:
# Проверяем распределение классов
def check_distribution(labels, dataset_name):
    unique, counts = np.unique(labels, return_counts=True)
    percentages = (counts / len(labels)) * 100
    print(f"\n{dataset_name}:")
    for cls, count, percent in zip(unique, counts, percentages):
        print(f"  Класс {cls}: {count} примеров ({percent:.1f}%)")

check_distribution(y_train, "TRAIN распределение")
check_distribution(y_val, "VAL распределение")


TRAIN распределение:
  Класс 0: 21676 примеров (25.1%)
  Класс 1: 5601 примеров (6.5%)
  Класс 2: 4461 примеров (5.2%)
  Класс 3: 3614 примеров (4.2%)
  Класс 4: 4221 примеров (4.9%)
  Класс 5: 4172 примеров (4.8%)
  Класс 6: 4953 примеров (5.7%)
  Класс 7: 5842 примеров (6.8%)
  Класс 8: 7377 примеров (8.5%)
  Класс 9: 24589 примеров (28.4%)

VAL распределение:
  Класс 0: 5419 примеров (25.1%)
  Класс 1: 1401 примеров (6.5%)
  Класс 2: 1115 примеров (5.2%)
  Класс 3: 903 примеров (4.2%)
  Класс 4: 1055 примеров (4.9%)
  Класс 5: 1043 примеров (4.8%)
  Класс 6: 1238 примеров (5.7%)
  Класс 7: 1461 примеров (6.8%)
  Класс 8: 1844 примеров (8.5%)
  Класс 9: 6148 примеров (28.4%)
